livree par client employee et date meme chose avec non livreee

# Livree

## Command + client

# Resultat + employee

# non Livree

# model pour represente un entropot projet te3na ndiro etoile

Etoile / constelation / flocan de naige 
Table de fait / Table de dimention
1/etile ( une seul table de fait central - pas de relation entre les tables de dimentions mais il ya entre fait et dimention)
pas de relation entre les dimentions alors abscence de hierachie entre les dim

etoile (simple) ---> flocan (une seul table de fait +dimention seoare+heiarchi entre dim +normalisation) ---> constelation(redondance plusieur fait dim commune entre fait+hearchie entre dimention) ( koul wa7ed rigla lo5er)

mesure = client , ordre , employee

dim (axe d'analise) = client , ordre , employee,temps (lazem dire dim temps )

multi dimention = donne de plusieur tables

Fait te3na hya command 3endha id howa id sequenciel ( num qui incremante) + concatination key primaire des dimentions id1id2...
+ attribute te3 command nbr commandes livree/non livree

# Cleaning ACCESS

- **Customers**

In [ ]:
for i in [customers,order_details,orders,products,purchase_orders,shippers,suppliers]:
        i.drop_duplicates(inplace=True)

#['Order Date'] = i['Order Date'].str.strip()

customers['ContactName'] = (
    customers['First Name'].fillna('') + ' ' + customers['Last Name'].fillna('')
)
customers['ContactName'] = customers['ContactName'].str.strip().str.replace(r'\s+', ' ', regex=True)
customers.drop(columns=['First Name', 'Last Name'], inplace=True)

customers.rename(columns={'Country/Region':'Country'},inplace=True)
customers.drop(columns=['E-mail Address','Job Title','Business Phone','Home Phone','Mobile Phone',
                        'Fax Number','Address','City','State/Province','ZIP/Postal Code','Web Page',
                        'Notes','Attachments'],inplace=True)

customers.to_csv('customer.csv', index=False)

- **orders**

In [ ]:
orders['Order Date'] = pd.to_datetime(orders['Order Date']).dt.strftime('%Y-%m-%d')
orders['Shipped Date'] = pd.to_datetime(orders['Shipped Date']).dt.strftime('%Y-%m-%d')

cols_to_drop = [
    'Ship Name', 'Ship Address', 'Ship City', 'Ship State/Province',
    'Ship ZIP/Postal Code', 'Ship Country/Region',
    'Taxes', 'Payment Type', 'Paid Date',
    'Notes', 'Tax Rate', 'Tax Status', 'Status ID'
]
orders.drop(columns=[c for c in cols_to_drop if c in orders.columns], inplace=True)




orders.columns = orders.columns.str.replace(' ','',regex = False)

orders.to_csv('order.csv', index=False)


- **Employee**

In [ ]:
if 'First Name' in employees.columns or 'Last Name' in employees.columns:
    employees['FullName'] = (
        employees.get('First Name', '').fillna('') + ' ' + employees.get('Last Name', '').fillna('')
    ).str.strip()
    # drop the original name columns if they exist
    employees.drop(columns=[c for c in ['First Name', 'Last Name'] if c in employees.columns], inplace=True)

cols_to_remove = [
    'E-mail Address','Job Title','Business Phone','Home Phone','Mobile Phone',
    'Fax Number','Address','City','State/Province','ZIP/Postal Code',
    'Country/Region','Web Page','Notes','Attachments'
]
employees.drop(columns=[c for c in cols_to_remove if c in employees.columns], inplace=True)

employees.to_csv('employees.csv', index=False)

# Cleaning SQL

In [ ]:
df_customers.rename(columns={'CompanyName':'Company'},inplace=True)
cols_to_drop = [
    'ContactTitle','Address','City','Region','PostalCode','Phone','Fax'
]
df_customers.drop(columns=[c for c in cols_to_drop if c in df_customers.columns], inplace=True)


# ANALYTICS

## 1- TRANSFORM WAREHOUSE INTO CSV FILES

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import urllib

# Encode the ODBC driver name
params = urllib.parse.quote_plus(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=localhost\\SQLEXPRESS;"
    "Database=Northwind_DW;"
    "Trusted_Connection=yes;"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")


dim_customer = pd.read_sql("SELECT * FROM DimCustomer", engine)
dim_employee = pd.read_sql("SELECT * FROM DimEmployee", engine)
dim_time = pd.read_sql("SELECT * FROM DimTime", engine)
order_fact = pd.read_sql("SELECT * FROM Order_Fact", engine)

# Export to CSV
dim_customer.to_csv("../Data/warehouse/DimCustomer.csv", index=False)
dim_employee.to_csv("../Data/warehouse/DimEmployee.csv", index=False)
dim_time.to_csv("../Data/warehouse/DimTime.csv", index=False)
order_fact.to_csv("../Data/warehouse/Order_Fact.csv", index=False)


## 2- TRANSFORM THE WAREHOUSE CSV FILES INTO SPECIFIC ANALYTICS / DASHBOARDS

In [22]:

import pandas as pd

customers = pd.read_csv("../Data/warehouse/DimCustomer.csv")
employees = pd.read_csv("../Data/warehouse/DimEmployee.csv")
orders = pd.read_csv("../Data/warehouse/Order_Fact.csv")
time_dim = pd.read_csv("../Data/warehouse/DimTime.csv")

customers.columns = customers.columns.str.strip()
employees.columns = employees.columns.str.strip()
orders.columns = orders.columns.str.strip()
time_dim.columns = time_dim.columns.str.strip()

orders['ShippedDateKey'] = orders['ShippedDateKey'].astype(int)

shipped_summary = pd.DataFrame({
    'ShippedStatus': ['Shipped', 'NotShipped'],
    'Count': [
        (orders['ShippedDateKey'] != 1011900).sum(),
        (orders['ShippedDateKey'] == 1011900).sum()
    ]
})
shipped_summary.to_csv("../Data/analytics/shipped_summary.csv", index=False)



shipped_by_employee = orders[orders['ShippedDateKey'] != 1011900]\
    .groupby('EmployeeID').size().reset_index(name='ShippedCount')
shipped_by_employee = shipped_by_employee.merge(employees[['EmployeeID','FullName']], on='EmployeeID')
shipped_by_employee.to_csv("../Data/analytics/shipped_by_employee.csv", index=False)


shipped_by_customer = orders[orders['ShippedDateKey'] != 1011900]\
    .groupby('CustomerID').size().reset_index(name='ShippedCount')
shipped_by_customer = shipped_by_customer.merge(customers[['CustomerID','Company']], on='CustomerID')
shipped_by_customer.to_csv("../Data/analytics/shipped_by_customer.csv", index=False)

shipped_by_country = orders[orders['ShippedDateKey'] != 1011900]\
    .merge(customers[['CustomerID','Country']], on='CustomerID')\
    .groupby('Country').size().reset_index(name='ShippedCount')
shipped_by_country.to_csv("../Data/analytics/shipped_by_country.csv", index=False)

shipped_by_company = orders[orders['ShippedDateKey'] != 1011900]\
    .merge(customers[['CustomerID','Company']], on='CustomerID')\
    .groupby('Company').size().reset_index(name='ShippedCount')
shipped_by_company.to_csv("../Data/analytics/shipped_by_company.csv", index=False)



# Not shipped by employee
not_shipped_by_employee = orders[orders['ShippedDateKey'] == 1011900]\
    .groupby('EmployeeID').size().reset_index(name='NotShippedCount')
not_shipped_by_employee = not_shipped_by_employee.merge(employees[['EmployeeID','FullName']], on='EmployeeID')
not_shipped_by_employee.to_csv("../Data/analytics/not_shipped_by_employee.csv", index=False)

# Not shipped by customer
not_shipped_by_customer = orders[orders['ShippedDateKey'] == 1011900]\
    .groupby('CustomerID').size().reset_index(name='NotShippedCount')
not_shipped_by_customer = not_shipped_by_customer.merge(customers[['CustomerID','Company']], on='CustomerID')
not_shipped_by_customer.to_csv("../Data/analytics/not_shipped_by_customer.csv", index=False)

# Not shipped by country
not_shipped_by_country = orders[orders['ShippedDateKey'] == 1011900]\
    .merge(customers[['CustomerID','Country']], on='CustomerID')\
    .groupby('Country').size().reset_index(name='NotShippedCount')
not_shipped_by_country.to_csv("../Data/analytics/not_shipped_by_country.csv", index=False)

# Not shipped by company
not_shipped_by_company = orders[orders['ShippedDateKey'] == 1011900]\
    .merge(customers[['CustomerID','Company']], on='CustomerID')\
    .groupby('Company').size().reset_index(name='NotShippedCount')
not_shipped_by_company.to_csv("../Data/analytics/not_shipped_by_company.csv", index=False)



shipped_orders = orders[orders['ShippedDateKey'] != 1011900]

# Merge with DimTime on ShippedDateKey
shipped_orders = shipped_orders.merge(
    time_dim[['DateKey', 'Year', 'Month']], 
    left_on='ShippedDateKey', 
    right_on='DateKey', 
    how='left'
)

shipped_per_year = shipped_orders.groupby('Year').size().reset_index(name='ShippedCount')
shipped_per_year = shipped_per_year.sort_values(by='ShippedCount', ascending=False)
shipped_per_year.to_csv("../Data/analytics/most_shipped_per_year.csv", index=False)

# Most shipped per month (across all years)
shipped_per_month = shipped_orders.groupby('Month').size().reset_index(name='ShippedCount')
shipped_per_month = shipped_per_month.sort_values(by='ShippedCount', ascending=False)
shipped_per_month.to_csv("../Data/analytics/most_shipped_per_month.csv", index=False)
